In [1]:
import pandas as pd
import re
from rusenttokenize import ru_sent_tokenize
import itertools
from pymorphy2 import MorphAnalyzer
import collections
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.corpus import stopwords

import pickle
import configparser
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
from tqdm import tqdm
import os

import configparser
import pandas as pd
from nltk.corpus import stopwords
from string import punctuation
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
import collections
import numpy as np
import pickle
from copy import deepcopy

from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import glob

import pandas as pd
import re
from rusenttokenize import ru_sent_tokenize
import itertools
from pymorphy2 import MorphAnalyzer
import collections
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
import nltk


morph = MorphAnalyzer()

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Делаем эмбеддинги статей 

## сначала получаем тексты статей

In [3]:
# dogovor = pd.read_excel(open('bow.xlsx', 'rb'), sheet_name='bow')
dogovor = pd.read_excel(open('../dog_data/bow.xlsx', 'rb'), sheet_name='bow')
dogovor.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,"Матрица ответственности, разграничения полномо...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Адресат в переписке/Уровень принятия Решения,NaN,NaN,NaN,NaN
2,№,Рамочный Договор № ГНЯ-19/11024/00865/Р/9-03/Г...,Заинтересованность,NaN,Ответственность участников на этапах реализаци...,NaN,NaN,Вернеуровневый (стратегический уровень),Владелец договора (тактический уровень),(операционный уровень),HSE,\nИсполнитель на месте
3,NaN,NaN,ЗАКАЗЧИК,ПОДРЯДЧИК,мобилизация/ПОДГОТОВКА,реализация/ВЫПОЛНЕНИЕ РАБОТ,Завершение работ/закрытие,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dogovor = pd.Series(dogovor['Unnamed: 1'], name='data')
dogovor = dogovor.dropna()
dogovor = dogovor.reset_index()
dogovor = dogovor.drop('index', axis=1)
dogovor

,data
0,Рамочный Договор № ГНЯ-19/11024/00865/Р/9-03/Г...
1,СТАТЬЯ 2. Предмет договора ( п.п. 2.1 ...
2,2.8. В целях своевременного выполнения...
3,СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 _...
4,3.1. Договорная цена формируется по До...
...,...
291,32.1. В случае изменений в цепочке собств...
292,32.2. Заказчик вправе в одностороннем пор...
293,СТАТЬЯ 33. ЗАКЛЮЧИТЕЛЬНЫЕ ПОЛОЖЕНИЯ ( п....
294,СТАТЬЯ 34. ПРИЛОЖЕНИЯ К ДОГОВОРУ:


In [5]:
def clean(text):
    text = text.replace('_', ' ')
    text = text.replace('\xa0', '')
    text = text.replace('–', '')
    text = text.replace('…', '')
    text = re.sub('[.]{2,}', '', text)
    text = text.strip()
    text = " ".join(text.split())

    return text

In [6]:
dogovor['data_clean'] = dogovor['data'].apply(lambda x: clean(x))
dogovor.head()

,data,data_clean
0,Рамочный Договор № ГНЯ-19/11024/00865/Р/9-03/Г...,Рамочный Договор № ГНЯ-19/11024/00865/Р/9-03/Г...
1,СТАТЬЯ 2. Предмет договора ( п.п. 2.1 ...,СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8)
2,2.8. В целях своевременного выполнения...,2.8. В целях своевременного выполнения Работ п...
3,СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 _...,СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)
4,3.1. Договорная цена формируется по До...,3.1. Договорная цена формируется по Договору в...


In [7]:
punkts = dogovor['data_clean'].tolist()
punkts = punkts[1:]
punkts

['СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8)',
 '2.8. В целях своевременного выполнения Работ по настоящему Договору, Заказчик предоставляет документацию согласно п.9.3 настоящего Договора.',
 'СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)',
 '3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет:',
 '3.1.1 По мере выхода рабочей документации формируются Наряд-заказы.',
 'Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору.',
 'Стоимость Договора включает в себя:',
 'стоимость Работ по Наряд-Заказам;',
 'стоимость поставки Оборудования и Материалов Подрядчика;',
 'резерв средств Заказчика, предназначенного для возмещения работ и затрат, потребность в которых возникает в ходе строительства в результате уточнения проектных решений и условий строительства;',
 'другие затраты Подрядчика, оп

In [8]:
all_articles = []
article = []
for i, line in enumerate(punkts):
    if line.startswith('СТАТЬЯ'):
        all_articles.append(article)
        article = []
        article.append(line)
        continue
    elif line.startswith('РАЗДЕЛ'):
        continue
    else:
        article.append(line)

In [9]:
all_articles = all_articles[1:]
# all_articles_without_25 = [article for article in all_articles if len(article)>1]
all_articles

[['СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8)',
  '2.8. В целях своевременного выполнения Работ по настоящему Договору, Заказчик предоставляет документацию согласно п.9.3 настоящего Договора.'],
 ['СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)',
  '3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет:',
  '3.1.1 По мере выхода рабочей документации формируются Наряд-заказы.',
  'Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору.',
  'Стоимость Договора включает в себя:',
  'стоимость Работ по Наряд-Заказам;',
  'стоимость поставки Оборудования и Материалов Подрядчика;',
  'резерв средств Заказчика, предназначенного для возмещения работ и затрат, потребность в которых возникает в ходе строительства в результате уточнения проектных решений и условий строительства;',
  'другие затраты По

In [10]:
united_articles = [' '.join(article) for article in all_articles]
united_articles

['СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8) 2.8. В целях своевременного выполнения Работ по настоящему Договору, Заказчик предоставляет документацию согласно п.9.3 настоящего Договора.',
 'СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20) 3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет: 3.1.1 По мере выхода рабочей документации формируются Наряд-заказы. Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору. Стоимость Договора включает в себя: стоимость Работ по Наряд-Заказам; стоимость поставки Оборудования и Материалов Подрядчика; резерв средств Заказчика, предназначенного для возмещения работ и затрат, потребность в которых возникает в ходе строительства в результате уточнения проектных решений и условий строительства; другие затраты Подрядчика, определяемые Сторонами в соответствии 

In [11]:
lens = [len(article.split()) for article in united_articles]
print(lens)
print(len(lens))

[24, 796, 526, 663, 430, 128, 899, 254, 372, 57, 39, 140, 464, 431, 123, 187, 36, 190, 129, 99, 245, 111, 1467, 8, 141, 144, 420, 48, 49, 124, 118, 8, 5]
33


In [12]:
sum(i > 350 for i in lens) 

10

In [13]:
stops = set(stopwords.words('russian'))
def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word not in stops]
    normalized_text = [word.replace('п.', '').replace('№', '') for word in normalized_text]
    normalized_text = [re.sub(r'\d+(?:\.\d+)?', '', word)  for word in normalized_text]
    normalized_text = ' '.join(normalized_text)
    normalized_text = re.sub(' +', ' ', normalized_text)
    
    return normalized_text

In [14]:
print(united_articles[5])
print(normalize(united_articles[5]))

СТАТЬЯ 7. ЗАВЕРЕНИЯ ОБ ОБСТОЯТЕЛЬСТВАХ, ИМЕЮЩИХ ЗНАЧЕНИЕ ДЛЯ ЗАКЛЮЧЕНИЯ, ИСПОЛНЕНИЯ ИЛИ ПРЕКРАЩЕНИЯ ДОГОВОРА ( п.п. 7.1 7.9) 7.1.7. Подрядчик, подписывая Перечень нарушений и штрафных санкций (Приложение №9 к Договору), ознакомлен с размерами штрафных санкций, подтверждает их соразмерность последствиям возможных нарушений. В случае наличия в условиях Договора и любых приложениях к нему противоречий о порядке/форме фиксации и размере штрафных санкций по тождественным нарушениям, Стороны договорились применять к таким взаимоотношениям положения Приложения №10 к Договору «Перечень нарушений и штрафных санкций за нарушения в области информационной безопасности, ППЭБ, ОТ (промышленная, пожарная, экологическая безопасность и охраны труда). 7.9. В случае если нарушение Подрядчиком условий настоящего Договора повлекло смерть работника (-ов) Подрядчика, Заказчика или третьего лица, Заказчик имеет право на односторонний отказ от исполнения Договора с возмещением Подрядчику стоимости фактически в

In [15]:
normalized_articles = [normalize(article) for article in united_articles]

In [16]:
lens = [len(article.split()) for article in normalized_articles]
print(lens)

[16, 579, 371, 491, 325, 92, 679, 189, 269, 42, 26, 103, 328, 314, 92, 138, 27, 140, 98, 80, 181, 80, 1068, 4, 102, 100, 294, 33, 32, 85, 79, 4, 3]


In [17]:
sum(i > 350 for i in lens)

5

In [18]:
index2article = {}
for i, article_tuple in enumerate(zip(all_articles, normalized_articles)):
    index2article.update({i+2: article_tuple})

In [19]:
del index2article[25]

In [20]:
index2article

{2: (['СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8)',
   '2.8. В целях своевременного выполнения Работ по настоящему Договору, Заказчик предоставляет документацию согласно п.9.3 настоящего Договора.'],
  'статья предмет договор п цель своевременный выполнение работа настоящий договор заказчик предоставлять документация согласно настоящий договор'),
 3: (['СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)',
   '3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет:',
   '3.1.1 По мере выхода рабочей документации формируются Наряд-заказы.',
   'Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору.',
   'Стоимость Договора включает в себя:',
   'стоимость Работ по Наряд-Заказам;',
   'стоимость поставки Оборудования и Материалов Подрядчика;',
   'резерв средств Заказчика, предназначенного для воз

### частотные слова

In [82]:
top_words = Counter()
for value in index2article.values():
    article = value[1]
    top_words.update(article.split())

In [83]:
top_words.most_common(10)

[('подрядчик', 263),
 ('заказчик', 210),
 ('работа', 196),
 ('договор', 171),
 ('выполнение', 101),
 ('приложение', 96),
 ('случай', 76),
 ('срок', 75),
 ('сторона', 68),
 ('наряд-заказ', 63)]

In [86]:
splitted = []
for value in index2article.values():
    article = value[1].split()
    splitted.append(article)
splitted

[['статья',
  'предмет',
  'договор',
  'п',
  'цель',
  'своевременный',
  'выполнение',
  'работа',
  'настоящий',
  'договор',
  'заказчик',
  'предоставлять',
  'документация',
  'согласно',
  'настоящий',
  'договор'],
 ['статья',
  'договорный',
  'цена',
  'п',
  'договорный',
  'цена',
  'формироваться',
  'договор',
  'соответствие',
  'приложение',
  'составлять',
  '.',
  'мера',
  'выход',
  'рабочий',
  'документация',
  'формироваться',
  'наряд-заказ',
  'стоимость',
  'договора/задание',
  'строительство',
  'являться',
  'ориентировочный',
  'состоять',
  'стоимость',
  'наряд-заказ',
  'подлежать',
  'корректировка',
  'посредством',
  'внесение',
  'изменение',
  'посредством',
  'заключение',
  'дополнительный',
  'соглашение',
  'договор',
  'стоимость',
  'договор',
  'включать',
  'стоимость',
  'работа',
  'наряд-заказ',
  'стоимость',
  'поставка',
  'оборудование',
  'материал',
  'подрядчик',
  'резерв',
  'средство',
  'заказчик',
  'предназначить',
  'возме

In [94]:
intersec = set(splitted[0]).intersection(*splitted)

In [95]:
intersec

{'статья'}

In [96]:
for word, count in top_words.items():
    if word in intersec:
        print(word, count)

статья 35


## теперь делаем эмбеддинги их и схороняем в пикл


In [39]:
config_path = "config.ini"
config = configparser.ConfigParser()
config.read(config_path, encoding='UTF-8')

bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = config['MandatoryRequirementClassifier']['bert_path']
m = build_model(bert_config)

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [97]:
index2emb = {}
for key, value in index2article.items():
    # print(len(value[1].split()))
    try:
        _, _, _, _, _, sent_mean_embs, _ = m([value[1]])
    except RuntimeError:
        article = ' '.join(value[1].split(' ')[:350])
        # print(len(article.split()))
        _, _, _, _, _, sent_mean_embs, _ = m([article])
    index2emb.update({key: sent_mean_embs})

In [98]:
filename = 'dog_embs.pkl'
outfile = open(os.path.join('../dog_data', filename), 'wb')
# outfile = open((filename), 'wb')
pickle.dump(index2emb, outfile)
outfile.close()

# теперь пишем штуку для сравнения эмбеддинга входящего текста и всех эмбеддингов статей

In [21]:
from nltk.tokenize import WordPunctTokenizer
stops = set(stopwords.words('russian'))
def normalize(text):
    normalized_text = text.replace('\n', ' ')
    normalized_text = re.sub('(?:(?:https?):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '', normalized_text)
    wordpunct_tokenize = WordPunctTokenizer().tokenize
    normalized_text = wordpunct_tokenize(normalized_text)
    normalized_text = [word.replace('№', '') for word in normalized_text]
    normalized_text = [re.sub(r'\d+(?:\.\d+)?', '', word)  for word in normalized_text]
    normalized_text = [morph.parse(word.lower())[0].normal_form for word \
                                        in normalized_text if word not in punctuation+'«»']
    normalized_text = [word.strip(punctuation) for word in normalized_text if word not in stops]
    normalized_text = ' '.join(normalized_text)
    normalized_text = re.sub(' +', ' ', normalized_text)
    
    return normalized_text

In [22]:
%%time
pages = convert_from_path('../correspondencesemanticanalysis/sample.pdf', 500) 
# pages = convert_from_path('sample.pdf', 500)
image_counter = 1
text_full = []

for page in pages: 
    text = str(((pytesseract.image_to_string(page, lang="rus+eng"))))
    image_counter += 1
    text_full.append(text)

CPU times: user 283 ms, sys: 426 ms, total: 709 ms
Wall time: 15.3 s


In [23]:
text = ' '.join(text_full)
text  ## !! Текст относится к разделу 3 (поставка материалов и оборудования, статьи 10-11-12)

'Акционерное общество Joint Stock Company $ р |\nТРЕСТ КОКСОХИММОНТАЖ TRUST KOKSOHIMMONTAZH =, ож\nРФ, г. Москва, 115035 Teli. (495) 953 22 68 kxm@kxm.ru tel: +7495 953 22 68 36/5 Kadashevskaya emb.,\nКадашевская наб., 36/5 факс: (495) 953 39 75 www.kxm.ru fax: +7495 953 5975 115035, Moscow, Russia\nКуда/ТО ООО «Газпромнефть-Развитие» ПЕ\nKomy/ ATT: Гродзь М.А. ОТ/ЕКОМ: Пеккера В.Б.\nЗИД по производственной Заместителя руководителя\nбезопасности (БГП) проекта АО «Трест\nssl ee Hh se el a cein agelesecl Коксохиммонтаж\nФак ЕАХ №.: Crp// Pages:\nКопия/ СС: [ara/ Date: 14.01.2020\nИсх. КеЁ №,: PLE Ha/ Your Ref No:\n\n \n\n— AS = se ee\n\nЕсли этот документ прошел некачественно, пожалуйста, сообщите нам по тел. 495 951 2575\nIf there are any problems with this transmission, please contact us tel. +7 495 951 2575\n\nУважаемый Максим Александрович!\n\nАкционерным обществом «Трест КХМ» в 2019 году был разработан и согласован План\n\nуправления договором в области производственной безопасности

In [24]:
start = 'Уважаемый'
end = 'С уважением'
start_i = text.find(start)
end_i = text.find(end)
text = text[start_i:end_i]
text

'Уважаемый Максим Александрович!\n\nАкционерным обществом «Трест КХМ» в 2019 году был разработан и согласован План\n\nуправления договором в области производственной безопасности на оказание услуг по\nкапитальному строительству ООО «ГПН-Развитие» в рамках договора ГНЯ-19-11024-00865-Р-9-03\n(период действия договора с 05.06.2019 по 26. 12.2021) на 2019 год.\n\nРазделом № 3 вышеуказанного договора предусмотрен план проактивных мероприятий,\n\nнаправленных на улучшение в области ПБ, который включал в себя приобретение в 4 кв. 2019 года:\n\n1. Съемных захватов ползункого типа для безопасного подъема (спуска) на высоту с\nприменением вертикальных анкерных линий:\n\nЗащитных колпачков для арматуры;\n\nМетеостанции;\n\nСигнальных конусов, уличных знаков безопасности;\n\nПереговорных устройств во взрывозащищённом исполнении (рации);\n\nГазанализаторов с определением наличия кислорода (%) и углеводорода (мг/м3);\nАвтоматических наружных дефибрилляторов:\n\nСтендов с паспортами объектов строите

In [25]:
norm_text = normalize(text)
norm_text

'уважаемый максим александр акционерный общество трест кхм год разработать согласовать план управление договор область производственный безопасность оказание услуга капитальный строительство ооо гпн развитие рамка договор гнуть р период действие договор год раздел вышеуказанный договор предусмотреть план проактивный мероприятие направить улучшение область пб который включать приобретение кв год съёмный захват ползункий тип безопасный подъём спуск высота применение вертикальный анкерный линия защитный колпачок арматура метеостанция сигнальный конус уличный знак безопасность переговорный устройство взрывозащищить исполнение рация газанализатор определение наличие кислород углеводород мг метр автоматический наружное дефибриллятор стенд паспорт объект строительство согласно бутилировать питьевой вода bo seth ls re настоящее время дать раздел оформить заявка дать указание соответствующий служба закупить доставить новопортовский нгкм следующий мтц защитный колпачок арматура диаметр — шт защи

In [99]:
pickle_path = '../dog_data/dog_embs.pkl'
infile = open(pickle_path, 'rb')
embs_dict = pickle.load(infile)
infile.close()

In [100]:
_, _, _, _, _, sent_mean_emb_tgt, _ = m(norm_text)

In [101]:
dic = {}
for key, value in embs_dict.items():
    score = np.inner(value, sent_mean_emb_tgt[0]) / (
                np.linalg.norm(value) * (np.linalg.norm(sent_mean_emb_tgt[0])))
    dic.update({score[0]: key})
od = collections.OrderedDict(sorted(dic.items(), reverse=True))
for key, value in od.items(): 
    print(key, '\t', value)
result = list(od.values())[0]

0.5009774 	 18
0.48623288 	 2
0.46989313 	 30
0.45984578 	 23
0.4234471 	 12
0.42102793 	 32
0.41498464 	 11
0.41329685 	 29
0.40803495 	 7
0.40569368 	 13
0.3986837 	 15
0.39344993 	 27
0.39250004 	 16
0.38928536 	 9
0.3804423 	 17
0.37983057 	 22
0.37903285 	 31
0.37799722 	 14
0.374588 	 3
0.37057245 	 8
0.36477566 	 4
0.36407244 	 20
0.36294228 	 6
0.35996214 	 19
0.35713783 	 10
0.35370627 	 5
0.34982026 	 26
0.34652933 	 21
0.33888173 	 24
0.33879304 	 28


In [102]:
index2article[result]

(['СТАТЬЯ 18. СКРЫТЫЕ РАБОТЫ ( п.п. 18.1 18.5)',
  '18.4. Акты освидетельствования скрытых работ, акты гидравлических и/или пневматических испытаний, промежуточной приемки выполненных работ и лабораторных исследований составляются Подрядчиком в 3-х экземплярах и подписываются Представителями Подрядчика и Заказчика.'],
 'статья скрытый работа п акт освидетельствование скрытый работа акт гидравлический и/ить пневматический испытание промежуточный приёмка выполнить работа лабораторный исследование составляться подрядчик -х экземпляр подписываться представитель подрядчик заказчик')

In [103]:
index2article[30]

(['СТАТЬЯ 30. СОДЕРЖАНИЕ ОБЪЕКТА ( пп. 30.1 30.4)',
  '30.4. Подрядчик не несет ответственность за причиненный Объекту ущерб, если он возник в результате форс-мажорных обстоятельств, указанных в ст.26 Договора. В этом случае Подрядчик на усмотрение и за счет средств Заказчика обязан восстановить Объект в сроки, согласованные Сторонами в дополнительном соглашении.'],
 'статья содержание объект пп подрядчик нести ответственность причинить объект ущерб возникнуть результат форс-мажорный обстоятельство указанный ст. договор это случай подрядчик усмотрение счёт средство заказчик обязать восстановить объект срок согласовать сторона дополнительный соглашение')

Фактически, самая высокоя косинусная близость у самых коротких пунктов.

## ElMO

In [27]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

PermissionDeniedError: /tmp/tfhub_modules/48ab951bfe35732722bd465a2822752b390006c5.lock.tmp68552d53cd894cf69406b959a68927d4; Permission denied

In [108]:
index2emb = {}
for key, value in index2article.items():
    embs_elmo = elmo([value[1].split()])
    index2emb.update({key: embs_elmo})

In [109]:
filename = 'dog_embs_elmo.pkl'
outfile = open(os.path.join('../dog_data', filename), 'wb')
pickle.dump(index2emb, outfile)
outfile.close()

In [110]:
pickle_path = '../dog_data/dog_embs_elmo.pkl'
infile = open(pickle_path, 'rb')
embs_dict = pickle.load(infile)
infile.close()

In [111]:
emb_tgt = elmo([norm_text.split()])

In [115]:
dic = {}
for key, value in embs_dict.items():
    score = np.inner(value, emb_tgt) / (
                np.linalg.norm(value) * (np.linalg.norm(emb_tgt)))
    dic.update({score[0][0]: key})
od = collections.OrderedDict(sorted(dic.items(), reverse=True))
for key, value in od.items(): 
    print(key, '\t', value)
result = list(od.values())[0]

0.7692651 	 15
0.75820184 	 4
0.7479828 	 14
0.7466483 	 9
0.7379581 	 3
0.7368371 	 8
0.7367033 	 11
0.73603594 	 10
0.7358768 	 19
0.7333185 	 5
0.72599083 	 22
0.7179106 	 13
0.7059584 	 20
0.7046946 	 21
0.7026205 	 6
0.6999484 	 24
0.6910368 	 31
0.6836262 	 7
0.6821507 	 17
0.6819726 	 18
0.67922175 	 32
0.66675943 	 23
0.6661665 	 28
0.63806164 	 12
0.6380208 	 16
0.61820257 	 30
0.6165288 	 26
0.6131247 	 27
0.5886763 	 29
0.573185 	 2


In [117]:
index2article[15][0]

['СТАТЬЯ 15. МОТИВАЦИЯ СОБЛЮДЕНИЯ ПОЛОЖЕНИЙ ДОГОВОРА В ОБЛАСТИ ПБ ( п.п. 15.1 15.18)',
 '15.1. В целях качественного выполнения работ по Договору Стороны согласовали, что исполнение Подрядчиком обязательств в области ПБ (Статья 14 Договора, Приложения №13 «Соглашение в области производственной безопасности») является существенным условием.',
 '15.2. Заказчик в целях мотивации исполнения Подрядчиком положений Договора в области ПБ и, как следствие, снижения риска инцидентов и аварий при выполнении работ, имеет право выплачивать Подрядчику дополнительное материальное поощрение за соблюдение требований Заказчика в области ПЭБ ОТ и ГЗ (далее Поощрение). Оценка выполнения Подрядчиком требований Заказчика в области ПБ происходит с использованием Ключевых Показателей Эффективности в области ПБ (далее КПЭ ПЭБ).',
 '15.4. В с момента подписания Договора Подрядчик совместно с Заказчиком разрабатывает План управления договором в области ПБ (в составе Приложения №8 «Перечень локальных нормативных 

## Tf-Idf clusters

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(normalized_articles)
words = vectorizer.get_feature_names()
# print("words", words)


n_clusters=30
number_of_seeds_to_try=10
max_iter = 300
number_of_process=2
model = KMeans(n_clusters=n_clusters, max_iter=max_iter, n_init=number_of_seeds_to_try, 
               n_jobs=number_of_process).fit(X)

labels = model.labels_
ordered_words = model.cluster_centers_.argsort()[:, ::-1]

print("centers:", model.cluster_centers_)
print("labels", labels)
print("intertia:", model.inertia_)

texts_per_cluster = numpy.zeros(n_clusters)
for i_cluster in range(n_clusters):
    for label in labels:
        if label == i_cluster:
            texts_per_cluster[i_cluster] += 1 

print("Top words per cluster:")
for i_cluster in range(n_clusters):
    print("Cluster:", i_cluster, "texts:", int(texts_per_cluster[i_cluster])),
    for term in ordered_words[i_cluster, :10]:
        print("\t"+words[term])

print("\n")
print("Prediction")

Y = vectorizer.transform([norm_text])
predicted_cluster = model.predict(Y)[0]
texts_per_cluster[predicted_cluster] += 1

print("Cluster:", predicted_cluster, "texts:", int(texts_per_cluster[predicted_cluster])),
for term in ordered_words[predicted_cluster, :10]:
    print("\t"+words[term])

words ['gps', 'абзац', 'аванс', 'авансирование', 'авансовый', 'авария', 'авторский', 'адрес', 'акт', 'аналогичный', 'арбитраж', 'аудит', 'базовый', 'балансовый', 'банк', 'банка', 'банковский', 'банкротство', 'безвозмездно', 'безвозмездный', 'безопасности', 'безопасность', 'безотзывный', 'безусловный', 'бенефициар', 'бумажный', 'бухгалтер', 'важность', 'вариант', 'ведение', 'ведомость', 'ведущий', 'вернуть', 'вести', 'вестник', 'весь', 'взаимный', 'взаимоотношение', 'взаиморасчёт', 'взрывопожароопасный', 'взыскание', 'взыскать', 'вид', 'вина', 'виновный', 'включать', 'включая', 'включение', 'владелец', 'власть', 'вместо', 'внесение', 'внести', 'внесудебный', 'вносить', 'внутренний', 'внутриобъектовый', 'возбуждение', 'возврат', 'возврата', 'возвратить', 'возвращать', 'возвращаться', 'возлагаться', 'возместить', 'возмещать', 'возмещаться', 'возмещение', 'возможно', 'возможный', 'возникать', 'возникновение', 'возникнуть', 'восстановить', 'вправе', 'временной', 'время', 'вследствие', 'всту

centers: [[0.         0.05420819 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.11468096 0.        ]
 [0.         0.         0.         ... 0.         0.04785007 0.        ]
 [0.         0.         0.         ... 0.         0.07211068 0.        ]]
labels [10 23 22 14  1  2 21 25  0  5 12 18 27  7 11 19 13 28 20 24  4 16  3  6
  8 26  9 15 29 17]
intertia: -7.771561172376096e-16
Top words per cluster:
Cluster: 0 texts: 1
	материал
	подрядчик
	оборудование
	заказчик
	поставка
	давальческий
	получить
	использовать
	качество
	возврат
Cluster: 1 texts: 1
	выполнение
	графика
	работа
	наряд
	заказ
	срок
	приложение
	событие
	график
	ключевой
Cluster: 2 texts: 1
	нарушение
	санкция
	штрафной
	договор
	безопасность
	размер
	иметь
	приложение
	подрядчик
	ознакомить
Cluster: 3 texts: 1
	подрядч

## Word2Vec усредненный

In [148]:
import gensim

In [149]:
model = gensim.models.KeyedVectors.load_word2vec_format('../dog_data/wv_model.bin', binary=True)

In [166]:
model

In [180]:
def avg_feature_vector(sentence, model):
    num_features = model.vector_size
    words = [w for w in sentence if w in model]
    lw = len(words)
    if lw == 0:
        return None
    feature_matrix = np.zeros((lw, num_features), dtype='float32')
    for i in list(range(lw)):
        word = words[i]
        feature_matrix[i, :] = model[word]
    feature_vec = np.average(feature_matrix, axis=0)
    
    return feature_vec

In [181]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [185]:
test = norm_text.split()
test = [word + '_' + morph.parse(word)[0].tag.POS for word in test if morph.parse(word)[0].tag.POS != None]
test = [word.replace('ADJF', 'ADJ').replace('INFN', 'VERB') for word in test]
test

['уважаемый_ADJ',
 'максим_NOUN',
 'александр_NOUN',
 'акционерный_ADJ',
 'общество_NOUN',
 'трест_NOUN',
 'год_NOUN',
 'разработать_VERB',
 'согласовать_VERB',
 'план_NOUN',
 'управление_NOUN',
 'договор_NOUN',
 'область_NOUN',
 'производственный_ADJ',
 'безопасность_NOUN',
 'оказание_NOUN',
 'услуга_NOUN',
 'капитальный_ADJ',
 'строительство_NOUN',
 'ооо_NOUN',
 'развитие_NOUN',
 'рамка_NOUN',
 'договор_NOUN',
 'гнуть_VERB',
 'период_NOUN',
 'действие_NOUN',
 'договор_NOUN',
 'год_NOUN',
 'раздел_NOUN',
 'вышеуказанный_ADJ',
 'договор_NOUN',
 'предусмотреть_VERB',
 'план_NOUN',
 'проактивный_ADJ',
 'мероприятие_NOUN',
 'направить_VERB',
 'улучшение_NOUN',
 'область_NOUN',
 'который_ADJ',
 'включать_VERB',
 'приобретение_NOUN',
 'год_NOUN',
 'съёмный_ADJ',
 'захват_NOUN',
 'ползункий_ADJ',
 'тип_NOUN',
 'безопасный_ADJ',
 'подъём_NOUN',
 'спуск_NOUN',
 'высота_NOUN',
 'применение_NOUN',
 'вертикальный_ADJ',
 'анкерный_ADJ',
 'линия_NOUN',
 'защитный_ADJ',
 'колпачок_NOUN',
 'арматура_

In [188]:
tgt_avg = avg_feature_vector(test, model)
tgt_avg

array([ 0.06674969,  0.01767568, -0.06857876, -0.34286895,  0.03210389,
        0.15835263, -0.02055055, -0.03577191, -0.05723298,  0.06822316,
       -0.03397519,  0.11379844,  0.018621  , -0.08968502, -0.0623069 ,
       -0.03892639,  0.00422416,  0.01590486,  0.15928578, -0.08992805,
       -0.14075254, -0.04812537, -0.00162367, -0.03268258, -0.05924881,
       -0.09573545,  0.08030318,  0.08172282, -0.06892369, -0.08611029,
        0.10408463,  0.01875423, -0.07587148, -0.01484837, -0.07691738,
       -0.0381008 , -0.05876874,  0.10294019,  0.03991571, -0.12314709,
       -0.00057838, -0.19036156,  0.00648298,  0.07549444, -0.01987754,
       -0.06792728,  0.1811174 , -0.07684846, -0.19545998, -0.06306943,
       -0.02464638, -0.02503074,  0.2257157 ,  0.02414501,  0.17652045,
        0.10033218, -0.07287497,  0.0748198 ,  0.00788477,  0.01054686,
       -0.01565311,  0.17143902,  0.02703563, -0.16358176,  0.20176269,
       -0.13713616,  0.0018049 , -0.03690236,  0.20835109, -0.06

In [190]:
index2emb = {}
for key, value in index2article.items():
    article = value[1].split()
    article = [word + '_' + morph.parse(word)[0].tag.POS for word in article if morph.parse(word)[0].tag.POS != None]
    article = [word.replace('ADJF', 'ADJ').replace('INFN', 'VERB') for word in article]
    
    index2emb.update({key: avg_feature_vector(article, model)})

In [187]:
from scipy.spatial import distance

In [191]:
dic = {}
for key, value in index2emb.items():
    similarity =  1 - distance.cosine(value, tgt_avg)
    # distances = spatial.distance.cdist(old_samples_vec, new_samples_vec, 'cosine')
    dic.update({similarity: key})
od = collections.OrderedDict(sorted(dic.items(), reverse=True))
for key, value in od.items(): 
    print(key, '\t', value)
result = list(od.values())[0]

0.8834712505340576 	 8
0.8802978992462158 	 9
0.8742141127586365 	 3
0.867396891117096 	 15
0.861954391002655 	 22
0.8572689294815063 	 20
0.8570343255996704 	 4
0.8568364977836609 	 10
0.856143057346344 	 14
0.8526010513305664 	 5
0.849604606628418 	 6
0.8483971357345581 	 13
0.8481302261352539 	 19
0.8452126979827881 	 24
0.8356494307518005 	 23
0.8353529572486877 	 31
0.8329482078552246 	 11
0.8310306072235107 	 16
0.8287608027458191 	 32
0.8262641429901123 	 7
0.8214143514633179 	 28
0.8203588128089905 	 21
0.8082736134529114 	 30
0.807916522026062 	 12
0.8018918037414551 	 18
0.7944614291191101 	 17
0.7789832353591919 	 29
0.7754091620445251 	 26
0.7686817646026611 	 27
0.7477598190307617 	 2


In [193]:
index2article[9][0]

['СТАТЬЯ 9. ОБЯЗАТЕЛЬСТВА ЗАКАЗЧИКА ( п.п. 9.1 9.15)',
 'По Договору Заказчик:',
 '9.1. Осуществляет оплату выполненных Подрядчиком Работ в порядке, предусмотренном ст. 5 Договора, а также производит иные платежи, предусмотренные Договором/Наряд-заказом(и) и/или Дополнительными соглашениями к нему.',
 '9.2. Обеспечивает получение и передачу Подрядчику необходимых документов, разрешений и согласований соответствующих органов власти, а именно:',
 'геодезическую разбивочную основу;',
 'согласование условий производства Работ в зоне действующих коммуникаций;',
 'технические условия на подключение к действующим инженерным коммуникациям на период выполнения Работ.',
 'разрешение на строительство.',
 '9.3. Передает Подрядчику 2 (два) экземпляра Проектной/Рабочей документации, утвержденной им «В производство работ».',
 'Подрядчик в течение получения Проектной/Рабочей документации направляет замечания, в случае отсутствия замечаний совместно с Заказчиком оформляет Протокол об отсутствии со стор